<a href="https://colab.research.google.com/github/raunakyadav92/DATASET_LEARNING/blob/main/DATASET_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"raunakyadav92","key":"d670cc702e0a4e98e4b8ac93899545f8"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install -q kaggle

In [4]:
!kaggle competitions download -c summer-school-hack-i

summer-school-hack-i.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q summer-school-hack-i.zip

replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/train.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:



from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
print("Shape:", df.shape)
print("\nMissing values:\n", df.isnull().sum())
print("\nData types:\n", df.dtypes)

In [ ]:
plt.figure(figsize=(15, 12))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

In [ ]:
sns.histplot(df['target'], kde=True)
plt.title("Target Distribution")
plt.show()

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:





scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:


lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_pred = lr.predict(X_test_scaled)

print("R2 Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
pca = PCA()
pca.fit(X_train_scaled)

In [ ]:

plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA - Explained Variance')
plt.grid()
plt.show()

In [ ]:


pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

lr_pca = LinearRegression()
lr_pca.fit(X_train_pca, y_train)

y_pred_pca = lr_pca.predict(X_test_pca)

print("PCA R2 Score:", r2_score(y_test, y_pred_pca))
print("PCA RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_pca)))

In [ ]:



selector = SelectKBest(score_func=f_regression, k=10)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

lr_fs = LinearRegression()
lr_fs.fit(X_train_selected, y_train)

y_pred_fs = lr_fs.predict(X_test_selected)

print("FS R2 Score:", r2_score(y_test, y_pred_fs))
print("FS RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_fs)))


In [ ]:
from sklearn.impute import SimpleImputer
from scipy.stats import zscore

In [ ]:
print("Shape:", df.shape)
print("\nMissing Values:\n", df.isnull().sum())
df.describe()

In [ ]:


z_scores = np.abs(zscore(X))
X_clean = X[(z_scores < 3).all(axis=1)]
y_clean = y[X_clean.index]

print("After removing outliers:", X_clean.shape)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_clean, test_size=0.2, random_state=42)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
X_df = pd.DataFrame(X_scaled, columns=[f'f{i}' for i in range(1, 21)])
vif_data = pd.DataFrame()
vif_data["Feature"] = X_df.columns
vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]

print(vif_data)

In [ ]:
!pip install xgboost catboost lightgbm optuna --quiet

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

print("Linear Regression R2 Score:", r2_score(y_test, y_pred_lr))
print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lr)))

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

print("XGBoost R2 Score:", r2_score(y_test, y_pred_xgb))
print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))

In [ ]:
from catboost import CatBoostRegressor

cat = CatBoostRegressor(verbose=0, random_state=42)
cat.fit(X_train, y_train)

y_pred_cat = cat.predict(X_test)

print("CatBoost R2 Score:", r2_score(y_test, y_pred_cat))
print("CatBoost RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_cat)))


In [ ]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(random_state=42)
lgbm.fit(X_train, y_train)

y_pred_lgbm = lgbm.predict(X_test)

print("LightGBM R2 Score:", r2_score(y_test, y_pred_lgbm))
print("LightGBM RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lgbm)))

In [ ]:
import optuna

def objective_xgb(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return mean_squared_error(y_test, pred)

In [ ]:

study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=50)

print("Best params for XGBoost:", study_xgb.best_params)
print("Best RMSE:", np.sqrt(study_xgb.best_value))

In [ ]:
best_xgb = XGBRegressor(**study_xgb.best_params)
best_xgb.fit(X_train, y_train)
y_pred_best = best_xgb.predict(X_test)

print("Final XGBoost R2:", r2_score(y_test, y_pred_best))
print("Final XGBoost RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_best)))

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

In [ ]:
def objective_stack(trial):
    xgb_params = {
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 8),
        'learning_rate': trial.suggest_float('xgb_lr', 0.01, 0.3),
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 500),
        'subsample': trial.suggest_float('xgb_subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('xgb_colsample', 0.6, 1.0),
        'random_state': 42
    }

    lgb_params = {
        'max_depth': trial.suggest_int('lgb_max_depth', 3, 8),
        'learning_rate': trial.suggest_float('lgb_lr', 0.01, 0.3),
        'n_estimators': trial.suggest_int('lgb_n_estimators', 100, 500),
        'subsample': trial.suggest_float('lgb_subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('lgb_colsample', 0.6, 1.0),
        'random_state': 42
    }

    cat_params = {
        'depth': trial.suggest_int('cat_depth', 3, 8),
        'learning_rate': trial.suggest_float('cat_lr', 0.01, 0.3),
        'iterations': trial.suggest_int('cat_iterations', 100, 500),
        'random_seed': 42,
        'verbose': 0
    }

    xgb = XGBRegressor(**xgb_params)
    lgb = LGBMRegressor(**lgb_params)
    cat = CatBoostRegressor(**cat_params)


    meta = Ridge()

    stack = StackingRegressor(
        estimators=[('xgb', xgb), ('lgb', lgb), ('cat', cat)],
        final_estimator=meta,
        passthrough=True,
        n_jobs=-1
    )

    stack.fit(X_train, y_train)
    preds = stack.predict(X_test)
    return mean_squared_error(y_test, preds)


In [ ]:
study_stack = optuna.create_study(direction='minimize')
study_stack.optimize(objective_stack, n_trials=25)

print("Best parameters from Optuna for StackingRegressor:")
for k, v in study_stack.best_params.items():
    print(f"{k}: {v}")

print("Best RMSE:", np.sqrt(study_stack.best_value))

In [ ]:

bp = study_stack.best_params

xgb_best = XGBRegressor(
    max_depth=bp['xgb_max_depth'],
    learning_rate=bp['xgb_lr'],
    n_estimators=bp['xgb_n_estimators'],
    subsample=bp['xgb_subsample'],
    colsample_bytree=bp['xgb_colsample'],
    random_state=42
)

lgb_best = LGBMRegressor(
    max_depth=bp['lgb_max_depth'],
    learning_rate=bp['lgb_lr'],
    n_estimators=bp['lgb_n_estimators'],
    subsample=bp['lgb_subsample'],
    colsample_bytree=bp['lgb_colsample'],
    random_state=42
)

cat_best = CatBoostRegressor(
    depth=bp['cat_depth'],
    learning_rate=bp['cat_lr'],
    iterations=bp['cat_iterations'],
    random_seed=42,
    verbose=0
)

meta_learner = Ridge()

final_stack = StackingRegressor(
    estimators=[('xgb', xgb_best), ('lgb', lgb_best), ('cat', cat_best)],
    final_estimator=meta_learner,
    passthrough=True,
    n_jobs=-1
)

final_stack.fit(X_train, y_train)
y_pred_stack = final_stack.predict(X_test)

print("Stacking Regressor R2:", r2_score(y_test, y_pred_stack))
print("Stacking Regressor RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_stack)))


In [ ]:
# submission = pd.DataFrame({"id": test_df["id"], "target": predictions})
# submission.to_csv("Submission.csv", index=False)
# print("Karman_Singh_2023362.csv created!")
